# セットアップ

処理が重いためGPU推奨です。


In [ ]:
!git clone https://github.com/anime-song/Chord-Transcription.git

In [ ]:
%cd /content/Chord-Transcription
!uv pip install -r requirements.txt
!uv pip install huggingface_hub --upgrade

# オーディオのアップロード

In [ ]:
from google.colab import files
uploaded = files.upload()  # choose a file
AUDIO_PATH = next(iter(uploaded.keys())) if uploaded else None
print('AUDIO_PATH:', AUDIO_PATH)

# セグメントモデルの実行 OptionA
モデルの実行はセグメントモデルとCRFモデルを選択できます。
CRFモデルのほうがより滑らかな出力となっているため、セグメントモデルはあくまで確認用です。


In [ ]:
%cd /content/Chord-Transcription

import os

from huggingface_hub import hf_hub_download

checkpoints_dir = 'checkpoints'
if not os.path.exists(checkpoints_dir):
    os.makedirs(checkpoints_dir)
    print(f'Created directory: {checkpoints_dir}')
else:
    print(f'Directory already exists: {checkpoints_dir}')

file_path = os.path.join(checkpoints_dir, 'base_model.pt')
if not os.path.exists(file_path):
    # モデルのダウンロード
    model_repo_id = 'anime-song/Chord-Transcription'
    model_filename = 'base_model.pt'

    print(f'Downloading {model_filename} from {model_repo_id}...')
    downloaded_path = hf_hub_download(
        repo_id=model_repo_id,
        filename=model_filename,
        local_dir=checkpoints_dir,
        local_dir_use_symlinks=False
    )
    print(f'Downloaded model to: {downloaded_path}')
else:
    print(f"File '{file_path}' already exists. Skipping download.")

!python -m src.inference --config ./configs/train.yaml --checkpoint ./checkpoints/base_model.pt \
  --audio $AUDIO_PATH

# CRFモデルの実行 OptionB

In [ ]:
%cd /content/Chord-Transcription

import os

from huggingface_hub import hf_hub_download

checkpoints_dir = 'checkpoints'
if not os.path.exists(checkpoints_dir):
    os.makedirs(checkpoints_dir)
    print(f'Created directory: {checkpoints_dir}')
else:
    print(f'Directory already exists: {checkpoints_dir}')

file_path = os.path.join(checkpoints_dir, 'crf_model.pt')
if not os.path.exists(file_path):
    # モデルのダウンロード
    model_repo_id = 'anime-song/Chord-Transcription'
    model_filename = 'crf_model.pt'

    print(f'Downloading {model_filename} from {model_repo_id}...')
    downloaded_path = hf_hub_download(
        repo_id=model_repo_id,
        filename=model_filename,
        local_dir=checkpoints_dir,
        local_dir_use_symlinks=False
    )
    print(f'Downloaded model to: {downloaded_path}')
else:
    print(f"File '{file_path}' already exists. Skipping download.")

!python -m src.inference --config ./configs/train.yaml --crf_checkpoint ./checkpoints/crf_model.pt \
  --use_segment_model --audio $AUDIO_PATH

# 結果のダウンロード


In [ ]:
from google.colab import files
import os

# AUDIO_PATHからファイル名を抽出 (拡張子なし)
base_name = os.path.splitext(os.path.basename(AUDIO_PATH))[0]

# 予測結果ファイルへのパスを構築
chords_file = f'predictions/{base_name}.chords.txt'
key_file = f'predictions/{base_name}.key.txt'
logits_image = f'predictions/{base_name}_chord25_logits.png'

# ZIPファイル名を設定
zip_filename = f'{base_name}_predictions.zip'

# 予測結果ファイルが存在するか確認し、存在するものだけをzipに追加
files_to_zip = []
if os.path.exists(chords_file): files_to_zip.append(chords_file)
if os.path.exists(key_file): files_to_zip.append(key_file)
if os.path.exists(logits_image): files_to_zip.append(logits_image)

if files_to_zip:
    # zipコマンドを実行
    !zip -j {zip_filename} {' '.join(files_to_zip)}
    print(f'Created {zip_filename}.')

    # ダウンロード
    files.download(zip_filename)
    print(f'Downloading {zip_filename}...')
else:
    print('No prediction files found to zip.')